In [386]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [387]:
test_data = pd.read_csv('test.csv')
test_data_org = test_data
train_data = pd.read_csv('train.csv')

In [388]:
#print(train_data.info()) #about data 
#Age, cabin, Embarked entries are not in all rows
#print(test_data.info()) #about test data 

In [389]:
#print(train_data.describe()) #only numerical fields

In [390]:
print(train_data.columns.values)
print(test_data.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']
['PassengerId' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare'
 'Cabin' 'Embarked']


In [391]:
#print(train_data)

In [392]:
def Visualize_Barchart(feature):
    survived = train_data[train_data['Survived']==1][feature].value_counts()
    print(survived)
    non_survived = train_data[train_data['Survived']==0][feature].value_counts()
    #print(non_survived)
    df = pd.DataFrame([survived, non_survived])
    #print(df)
    df.index = ['Survived', 'Non_Survived']
    df.plot(kind='bar', stacked=True, figsize=(10, 5))

In [393]:
def Summarize_count(feature):
    print(feature)
    survived = train_data[train_data['Survived']==1][feature].value_counts()
    #print(survived)
    non_survived = train_data[train_data['Survived']==0][feature].value_counts()
    #print(non_survived)
    df = pd.DataFrame([survived, non_survived])
    #print(df)
    df.index = ['Survived', 'Non_Survived']
    print(df)
    #pd.pivot_table(train_data, index = 'feature', values = [survived, non_survived], aggfunc="count")

In [394]:
''''
Summarize_count('Sex')
print()
Summarize_count('Age')
print()
Summarize_count('SibSp')
print()
Summarize_count('Pclass')
print()
Summarize_count('Fare')
#pd.pivot_table(train_data, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])
#pd.pivot_table(train_data, index = 'SibSp', values = ['Survived'], aggfunc="count")


SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-394-f9aa5bb37b78>, line 12)

In [395]:
#Visualize_Barchart(['Sex','Fare'])
#Visualize_Barchart('Embarked')
#Visualize_Barchart('Cabin')
#Visualize_Barchart('Fare')
#Visualize_Barchart('Pclass')
#Visualize_Barchart(['SibSp', 'Parch'])
#Visualize_Barchart('SibSp')
#Visualize_Barchart('Fare')
#Visualize_Barchart('Pclass')


In [396]:
train_data['family_count'] = train_data['SibSp'] + train_data['Parch'] + 1
test_data['family_count'] = test_data['SibSp'] + test_data['Parch'] + 1
#test_data['isAlone'] = [train_data['family_count'] == 0]
print(train_data.groupby('family_count')[['Survived']].agg('sum'))

              Survived
family_count          
1                  163
2                   89
3                   59
4                   21
5                    3
6                    3
7                    4
8                    0
11                   0


In [397]:
import numpy as np
#train_data["isAlone"] = train_data['family_count'].map(lambda f: 1 if f ==1 else 0)
#train_data["smallFamily"] = train_data['family_count'].map(lambda f: 1 if 2<= f <=4 else 0)
#train_data["bigFamily"] = train_data['family_count'].map(lambda f: 1 if 5<=f else 0)


#test_data["isAlone"] = test_data['family_count'].map(lambda f: 1 if f ==1 else 0)
#test_data["smallFamily"] = test_data['family_count'].map(lambda f: 1 if 2<= f <=4 else 0)
#test_data["bigFamily"] = test_data['family_count'].map(lambda f: 1 if 5<=f else 0)


In [398]:
#test_data['isAlone'] = [train_data['family_count'] == 0]
family_map = {1: 'Alone', 2: 'Small', 3: 'Small', 4: 'Small', 5: 'Medium', 6: 'Medium', 7: 'Large', 8: 'Large', 11: 'Large'}
train_data['family_Size_Grouped'] = train_data['family_count'].map(family_map)
test_data['family_Size_Grouped'] = test_data['family_count'].map(family_map)


In [399]:
#Extract title from Name
#print(pd.unique(train_data['Name']))

#categories 
titlemap_dict = {
    'Mr': 'Mr', 'Mrs':'Mrs', 'Miss': 'Miss', 'Master': 'Master', 'Don':'Royalty', 'Rev':'Officer', 'Dr':'Officer', 
    'Mme': 'Mrs', 'Ms': 'Mrs', 'Major':'Officer', 'Lady': 'Royalty', 'Sir': 'Royalty', 'Mlle':'Miss',
    'Col':'Officer', 'Capt':'Officer', 'the Countess':'Royalty', 'Jonkheer':'Royalty'}

train_data['title'] = train_data['Name'].apply(lambda n:n.split(',')[1].split('.')[0].strip())
train_data['title'] = train_data.title.map(titlemap_dict)


test_data['title'] = test_data['Name'].apply(lambda n:n.split(',')[1].split('.')[0].strip())
test_data['title'] = test_data.title.map(titlemap_dict)

print(pd.unique(train_data['title']))

['Mr' 'Mrs' 'Miss' 'Master' 'Royalty' 'Officer']


In [400]:
# Apply Age to missing fields based on group of instances
grouped_Age_Fields = train_data.groupby(['Sex', 'Pclass', 'title'])
grouped_Age_Fields = grouped_Age_Fields.median()
grouped_Age_Fields_median = grouped_Age_Fields.reset_index()[['Sex', 'Pclass', 'title', 'Age']]
grouped_Age_Fields_median.head()

def fill_age(row):
    c = (
        (grouped_Age_Fields_median['Sex'] == row['Sex']) &
        (grouped_Age_Fields_median['Pclass'] == row['Pclass']) &
        (grouped_Age_Fields_median['title'] == row['title'])
    )
    
    print(c) 
    
    if np.isnan(grouped_Age_Fields_median[c]['Age'].values[0]):
        print('true')
        c = ( 
            (grouped_Age_Fields_median['Sex'] == row['Sex']) &
            (grouped_Age_Fields_median['Pclass'] == row['Pclass'])
        )
        
    return grouped_Age_Fields_median[c]['Age'].values[0]

train_data['Age'] = train_data.apply(lambda r: fill_age(r) if np.isnan(r['Age']) else r['Age'], axis=1)
#print(train_data['Age'])

test_data['Age'] = test_data.apply(lambda r: fill_age(r) if np.isnan(r['Age']) else r['Age'], axis=1)
#print(test_data['Age'])
        

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13     True
14    False
15    False
16    False
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7      True
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10    False
11  

dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7      True
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10  

dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13     True
14    False
15    False
16    False
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7      True
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10  

dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1      True
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13     True
14    False
15    False
16    False
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10  

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11  

dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15     True
16    False
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16     True
dtype: bool
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7      True
8     False
9     False
10  

In [401]:
#fill Age with mean 
from sklearn.impute import SimpleImputer


#Fare as test data doesn't have info

imputer_fare = SimpleImputer(missing_values = np.nan, strategy="median") #median
imputer_fare.fit(train_data[['Fare']])

train_data['Fare'] = imputer_fare.transform(train_data[['Fare']])
test_data['Fare'] = imputer_fare.transform(test_data[['Fare']])

train_data['Embarked'].fillna('S', inplace = True)
test_data['Embarked'].fillna('S', inplace = True)

train_data['Cabin'].fillna('T', inplace = True)
train_data['Cabin'] = train_data['Cabin'].map(lambda c: c[0]) # map with first letter

test_data['Cabin'].fillna('T', inplace = True)
test_data['Cabin'] = test_data['Cabin'].map(lambda c: c[0]) # map with first letter

train_data['title'].fillna('None', inplace = True)
test_data['title'].fillna('None', inplace = True)oka

#imputer = SimpleImputer(missing_values = np.nan, strategy='mean')
#imputer.fit(train_data[['Age']])

#train_data['Age'] = imputer.transform(train_data[['Age', 'Pclass', 'title', 'Sex']])
#test_data['Age'] = imputer.transform(test_data[['Age', 'Pclass', 'title', 'Sex']])


print(train_data)
#x = x.fillna(df[['Age']].mean())




     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [402]:
y = train_data['Survived']
#train_data = train_data.drop(['Survived', 'PassengerId', 'Name', 'Cabin', 'Ticket', 'SibSp', 'Parch'], axis=1)
#test_data = test_data.drop(['PassengerId', 'Name', 'Cabin', 'Ticket', 'SibSp', 'Parch'], axis=1)

In [403]:
print(train_data.info()) #about data 
#Age, cabin, Embarked entries are not in all rows
print(test_data.info()) #about test data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PassengerId          891 non-null    int64  
 1   Survived             891 non-null    int64  
 2   Pclass               891 non-null    int64  
 3   Name                 891 non-null    object 
 4   Sex                  891 non-null    object 
 5   Age                  891 non-null    float64
 6   SibSp                891 non-null    int64  
 7   Parch                891 non-null    int64  
 8   Ticket               891 non-null    object 
 9   Fare                 891 non-null    float64
 10  Cabin                891 non-null    object 
 11  Embarked             891 non-null    object 
 12  family_count         891 non-null    int64  
 13  family_Size_Grouped  891 non-null    object 
 14  title                891 non-null    object 
dtypes: float64(2), int64(6), object(7)
memor

In [404]:
train_data.drop(columns=['Name', 'Ticket', 'Survived'], inplace=True)
test_data.drop(columns=['Name', 'Ticket'], inplace=True)


In [405]:
#dummpies = pd.get_dummies(train_data.Sex, drop_first=True)
train_data = pd.get_dummies(train_data, columns=['Sex', 'Embarked', 'Cabin', 'title', 'family_Size_Grouped'], drop_first=True)
test_data = pd.get_dummies(test_data, columns=['Sex', 'Embarked', 'Cabin', 'title', 'family_Size_Grouped'], drop_first=True)


In [406]:
print(train_data.head(5))
print(test_data.head(5))
#print(train_data.)


   PassengerId  Pclass   Age  SibSp  Parch     Fare  family_count  Sex_male  \
0            1       3  22.0      1      0   7.2500             2         1   
1            2       1  38.0      1      0  71.2833             2         0   
2            3       3  26.0      0      0   7.9250             1         0   
3            4       1  35.0      1      0  53.1000             2         0   
4            5       3  35.0      0      0   8.0500             1         1   

   Embarked_Q  Embarked_S  ...  Cabin_G  Cabin_T  title_Miss  title_Mr  \
0           0           1  ...        0        1           0         1   
1           0           0  ...        0        0           0         0   
2           0           1  ...        0        1           1         0   
3           0           1  ...        0        0           0         0   
4           0           1  ...        0        1           0         1   

   title_Mrs  title_Officer  title_Royalty  family_Size_Grouped_Large  \
0      

In [407]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_data[['Age', "Fare"]] = sc.fit_transform(train_data[['Age', "Fare"]])
test_data[['Age', "Fare"]] = sc.transform(test_data[['Age', "Fare"]])

In [408]:
print(train_data.shape, test_data.shape, y.shape)
#print(x_train.shape, test_data.shape, y.shape)
#print(train_data.head(5))
print(test_data.head(5))

(891, 25) (418, 25) (891,)
   PassengerId  Pclass       Age  SibSp  Parch      Fare  family_count  \
0          892       3  0.397533      0      0 -0.490783             1   
1          893       3  1.324310      1      0 -0.507479             2   
2          894       2  2.436442      0      0 -0.453367             1   
3          895       3 -0.158534      0      0 -0.474005             1   
4          896       3 -0.529244      1      1 -0.401017             3   

   Sex_male  Embarked_Q  Embarked_S  ...  Cabin_G  Cabin_T  title_Miss  \
0         1           1           0  ...        0        1           0   
1         0           0           1  ...        0        1           0   
2         1           1           0  ...        0        1           0   
3         1           0           1  ...        0        1           0   
4         0           0           1  ...        0        1           0   

   title_Mr  title_Mrs  title_None  title_Officer  family_Size_Grouped_Large  \
0  

In [409]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, y, test_size = 0.2, random_state = 1)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(712, 25) (179, 25) (712,) (179,)


In [410]:
print(test_data.shape)
print(train_data.shape)

#print(test_data)
x_train = train_data
y_train = y

print(test_data.shape)
print(train_data.shape)


(418, 25)
(891, 25)
(418, 25)
(891, 25)


In [411]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


In [412]:
#x_train = train_data
#y_train = y

In [413]:
dt_clf = DecisionTreeClassifier(random_state = 42)
cv = cross_val_score(dt_clf, x_train, y_train, cv=5)
print(cv)
print(cv.mean())

[0.60335196 0.76404494 0.78651685 0.79775281 0.83707865]
0.7577490427468458


In [414]:
rnd_clf = RandomForestClassifier(n_estimators=180, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
rnd_clf.fit(x_train, y_train)
print(rnd_clf.score(x_train, y_train))
y_pred = rnd_clf.predict(x_test)
rnd_clf_test_pred = rnd_clf.predict(test_data)
#cv = cross_val_score(rnd_clf, x_train, y_train, cv=5)
#print(cv)
#print(cv.mean())


0.9169472502805837


In [415]:

random_forest = RandomForestClassifier(criterion='gini',
                                           n_estimators=1750,
                                           max_depth=7,
                                           min_samples_split=6,
                                           min_samples_leaf=6,
                                           max_features='auto',
                                           oob_score=True,
                                           random_state=42,
                                           n_jobs=-1,
                                           verbose=1) 
random_forest.fit(train_data, y)
Y_prediction = random_forest.predict(test_data)



random_forest.score(train_data, y)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 1750 out of 1750 | elapsed:    2.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 1750 out of 1750 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent worker

0.8597081930415263

In [416]:
print(Y_prediction.shape)
print(test_data.shape)

(418,)
(418, 25)


In [417]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_prediction, test_data))

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [418]:
log_clf = LogisticRegression(max_iter=2000, random_state=42) #(solver="lbfgs", random_state=42)
#log_clf.fit(x, y)
#log_clf.predict(y_test)
cv = cross_val_score(log_clf, x_train, y_train, cv=5)
print(cv)
print(cv.mean())

[0.82122905 0.83707865 0.81460674 0.8258427  0.85393258]
0.8305379448873266


In [419]:
#svm_clf = SVC(gamma="scale", random_state=42)
#svm_clf.fit(x_train, y_train)
#y_pred = svm_clf.predict(x_test)
#print(y_pred)
#print(y_test)

svm_clf = SVC(probability = True, random_state=42)

cv = cross_val_score(svm_clf, x_train, y_train, cv=5)
print(cv)
print(cv.mean())
#svm_clf.fit(x_train, y_train)
#svm_clf_pred = svm_clf.predict(test_data)

[0.61452514 0.61797753 0.61797753 0.61797753 0.61235955]
0.6161634548992531


In [420]:
classifier = SVC(kernel = 'rbf', random_state = 0)
cv = cross_val_score(classifier, x_train, y_train, cv=5)
print(cv)
print(cv.mean())




[0.61452514 0.61797753 0.61797753 0.61797753 0.61235955]
0.6161634548992531


In [421]:
classifier.fit(x_train, y_train)
Y_pred = classifier.predict(test_data)

In [422]:
#svm_clf.fit(x_train, y_train)
#svm_clf.predict(test_data)

cv = cross_val_score(classifier, x_train, y_train, cv=5)
print(cv)
print(cv.mean())

[0.61452514 0.61797753 0.61797753 0.61797753 0.61235955]
0.6161634548992531


In [423]:
from sklearn.ensemble import BaggingClassifier
bag_clf = BaggingClassifier ( DecisionTreeClassifier (), n_estimators = 500 , max_samples = 100 , bootstrap = True , 
                             n_jobs =-1, oob_score=True ) 
bag_clf . fit ( x_train , y_train ) 
print(bag_clf.oob_score_)
bag_clf_y_pred = bag_clf . predict ( test_data ) 




0.8338945005611672


In [424]:
bag_auto_clf = BaggingClassifier ( 
    DecisionTreeClassifier 
    ( max_features = "auto" , max_leaf_nodes = 16 ), n_estimators = 500 ,  
    bootstrap = True , oob_score=True, n_jobs =-1 ) 
cv = cross_val_score(bag_auto_clf, x_train, y_train, cv=5)
print(cv)
print(cv.mean())

bag_auto_clf.fit(x_train, y_train)
print(bag_auto_clf.oob_score_)


[0.83798883 0.82022472 0.82022472 0.79775281 0.85393258]
0.8260247316552632
0.8316498316498316


In [558]:
#from sklearn.metrics import accuracy_score

#accuracy_score(test_data, )

#gbm_params= {"learning_rate":[0.1,0.01,0.001],
             "n_estimators": [500,1000],
             "min_samples_split": [2,4],
             "min_samples_leaf": [2,4],
             "max_depth": [3,6,15]}
#gbm_cv= GridSearchCV(GradientBoostingClassifier(),gbm_params,cv=10,n_jobs=-1,scoring="accuracy").fit(x_train,y_train)
#print('done')
#print(gbm_cv)

IndentationError: unexpected indent (<ipython-input-558-1e0375e91b05>, line 6)

In [425]:
gbm_tuned= GradientBoostingClassifier(learning_rate=0.01,max_depth=3,min_samples_leaf=2,
                                      min_samples_split=2,n_estimators=1500).fit(x_train,y_train)
cv = cross_val_score(gbm_tuned, x_train, y_train, cv=5)
print(cv)
print(cv.mean())

y_tuned_gbm= gbm_tuned.predict(test_data)
#accuracy_score(y_test,y_tuned_gbm)

[0.62011173 0.80337079 0.84269663 0.80898876 0.83707865]
0.7824493126608499


In [426]:
#rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
#log_clf = LogisticRegression(solver="lbfgs", random_state=42)

voting_clf_hard = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('scv', svm_clf), ('dt', dt_clf), ('bg', bag_clf), ('gbc', gbm_tuned)], voting='hard')
voting_clf_soft = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('scv', svm_clf), ('dt', dt_clf), ('bg', bag_clf), ('gbc', gbm_tuned)], voting='soft')


In [427]:

cv = cross_val_score(voting_clf_hard, x_train, y_train, cv=5)
print(cv)
print(cv.mean())

cv = cross_val_score(voting_clf_soft, x_train, y_train, cv=5)
print(cv)
print(cv.mean())

[0.62569832 0.82022472 0.83707865 0.80898876 0.86516854]
0.79143179963593
[0.67597765 0.81460674 0.83707865 0.81460674 0.85955056]
0.8003640700520996


In [399]:
voting_clf_hard.fit(x_train, y_train)
Y_pred_hard = voting_clf_hard.predict(test_data)

voting_clf_soft.fit(x_train, y_train)
Y_pred_soft = voting_clf_soft.predict(test_data)
#print(y_pred)
#print(y_test)



In [400]:
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
#df = pd.DataFrame(columns = ['Column_name#1', 'Column_name#2'])
#df['Column_name#1'] = y_test
#df['Column_name#2'] = Y_pred_soft
#print(df.head(20))

In [430]:
output = pd.DataFrame({'PassengerId': test_data_org.PassengerId, 'Survived': rnd_clf_test_pred})
output.to_csv('my_submission_random1.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
